# PointNet Element Parameter Detection

## Setup

In [ ]:
import numpy as np
import math
import random
import os
import json
import torch
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import plotly.graph_objects as go
import plotly.express as px

import ifcopenshell
from utils.JupyterIFCRenderer import JupyterIFCRenderer
from path import Path
import open3d as o3d

from src.elements import *

In [ ]:
# !pip install path.py;
# from path import Path

In [ ]:
# !pip install open3d
# import open3d as o3d

In [ ]:
random.seed = 42

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#path = Path("ModelNet10")
#path = Path('/content/drive/MyDrive/ElementNet/')
path = Path('output/')

In [ ]:
#savepath = '/content/drive/MyDrive/ElementNet/'
savepath = 'models/'


In [ ]:
def read_pcd(file):
    pcd = o3d.io.read_point_cloud(str(file))
    return np.asarray(pcd.points)

In [ ]:
f = path/"elbow/test/19024.pcd"
pointcloud = read_pcd(f)

In [ ]:
len(pointcloud)

### Initial viusalisation

In [ ]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [ ]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()
    

## Transforms

In [ ]:
pcshow(*pointcloud.T)

### Normalize

Unit sphere

In [ ]:
class Normalize(object):
    def __call__(self, data):
        pointcloud, properties = data[0], data[1]
        assert len(pointcloud.shape)==2
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_factor = np.max(np.linalg.norm(norm_pointcloud, axis=1))
        norm_pointcloud /= norm_factor
        properties_norm = properties/norm_factor
        #print(properties,properties_norm)

        return  (norm_pointcloud, properties_norm)

In [ ]:
dummy_properties = np.array([1.1, 2.2])

In [ ]:
 norm_pointcloud,_ = Normalize()((pointcloud, dummy_properties))

In [ ]:
pcshow(*norm_pointcloud.T)

### Augmentations

random rotation and random noise 

In [ ]:
class RandRotation_z(object):
    def __call__(self, data):
        pointcloud, properties = data[0], data[1]
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  (rot_pointcloud, properties)
    
class RandomNoise(object):
    def __call__(self, data):
        pointcloud, properties = data[0], data[1]
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  (noisy_pointcloud, properties)

In [ ]:
rot_pointcloud, _ = RandRotation_z()((norm_pointcloud, dummy_properties))
noisy_rot_pointcloud, _ = RandomNoise()((rot_pointcloud, dummy_properties))

In [ ]:
pcshow(*noisy_rot_pointcloud.T)

In [ ]:
class ToTensor(object):
    def __call__(self, data):
        pointcloud, properties = data[0], data[1]
        assert len(pointcloud.shape)==2

        return (torch.from_numpy(pointcloud).float(), torch.from_numpy(properties).float())

In [ ]:
ToTensor()((noisy_rot_pointcloud, dummy_properties))

## Dataset

In [ ]:
def default_transforms():
    return transforms.Compose([
                                Normalize(),
                                ToTensor()
                              ])

In [ ]:
def parse_pipe_properties(element_data):
  #target = [element_data['radius']/1000, element_data['length']/1000]
  scaled_targets = [element_data['radius']/1000, element_data['length']/1000]
  unscaled_targets = [element_data['direction'][0], element_data['direction'][1],
                      element_data['direction'][2]]
  #target = [element_data['radius']/1000]
  return np.array(scaled_targets), np.array(unscaled_targets)


In [ ]:
def parse_elbow_properties(element_data):
  #target = [element_data['radius']/1000, element_data['length']/1000]
  scaled_targets = [element_data['radius']/1000, element_data['axis_x']/1000, 
                    element_data['axis_y']/1000]
  unscaled_targets = [element_data['direction'][0], element_data['direction'][1], 
                      element_data['direction'][2],  element_data['angle']/200]
  #target = [element_data['radius']/1000]
  return np.array(scaled_targets), np.array(unscaled_targets)


In [ ]:
# scaled properties must be transformed when the cloud's scale is transformed
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", category='pipe', transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.category = category
        self.transforms = transform if not valid else default_transforms()
        metadata_file = open(root_dir/Path(category)/"metadata_new.json", 'r')
        metadata = json.load(metadata_file)
        self.valid = valid
        self.files = []

        new_dir = root_dir/Path(category)/folder
        for file in os.listdir(new_dir):
            if file.endswith('.pcd'):
                sample = {}
                sample['pcd_path'] = new_dir/file
                sample['id'] = int(file.split(".")[0])
                if category == 'pipe':
                    sample['scaled_properties'], sample['unscaled_properties'] = parse_pipe_properties(
                        metadata[file.split(".")[0]])
                elif category == 'elbow':
                    sample['scaled_properties'], sample['unscaled_properties'] = parse_elbow_properties(
                        metadata[file.split(".")[0]])
                self.files.append(sample)
        self.targets = len(self.files[0]['scaled_properties']) + len(
            self.files[0]['unscaled_properties'])

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file, properties):
        cloud = read_pcd(file)
        if self.transforms:
            pointcloud, properties = self.transforms((cloud, properties))
        return pointcloud, properties

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        scaled_properties = self.files[idx]['scaled_properties']
        unscaled_properties = torch.from_numpy(self.files[idx]['unscaled_properties']).float()
        id = self.files[idx]['id']
        pointcloud, scaled_properties = self.__preproc__(pcd_path, scaled_properties)
        return {'pointcloud': pointcloud, 
                'properties': torch.cat((scaled_properties, unscaled_properties)),
                'id': id}

Transforms for training. 1024 points per cloud as in the paper!

In [ ]:
train_transforms = transforms.Compose([
                    Normalize(),
#                    RandomNoise(),
                    ToTensor()
                    ])

In [ ]:
cat= 'elbow'
train_ds = PointCloudData(path, category=cat, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', category=cat, transform=train_transforms)

In [ ]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
#print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'])
print('Sample pointcloud label: ', train_ds[0]['properties'])
print('Sample pointcloud label: ', train_ds[0]['properties'])
#print('Class: ', inv_classes[train_ds[0]['category']])

In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

## Model

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix


class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, outputs = 2):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, outputs)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)


    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return output, matrix3x3, matrix64x64

In [ ]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.MSELoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

## Training loop

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
targets = train_ds.targets
pointnet = PointNet(outputs=targets)
pointnet.to(device);
#pointnet = pointnet.double()


In [ ]:
optimizer = torch.optim.Adam(pointnet.parameters(), lr=0.005)
criterion = torch.nn.MSELoss()

In [ ]:
def train(model, savepath, targets, train_loader, val_loader=None,  epochs=50, save=True):
    for epoch in range(epochs): 
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data['pointcloud'].to(device).float(), data['properties'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 10 == 9:    # print every 10 mini-batches
                    print('[Epoch: %d, Batch: %4d / %4d], loss: %.3f' %
                        (epoch + 1, i + 1, len(train_loader), running_loss / 10))
                    running_loss = 0.0

        pointnet.eval()
        total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                total_val_loss = np.zeros(targets)
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['properties'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    total += labels.size(0)
                    for i in range(targets):
                      val_loss = criterion(outputs[:,i], labels[:,i])
                      total_val_loss[i] += val_loss.item()
                    # _, predicted = torch.max(outputs.data, 1)
                    # 
                    # correct += (predicted == labels).sum().item()
            total_val_loss = total_val_loss / total
            print('Valid loss: ', total_val_loss.tolist())

        # save the model
        if save:
            torch.save(pointnet.state_dict(), savepath +"save_"+str(epoch)+".pth")

In [ ]:
#train(pointnet, savepath, targets, train_loader, valid_loader,  save=True)

## Test

Analyze results statistically

In [ ]:
pointnet = PointNet(targets)
#pointnet.load_state_dict(torch.load(savepath +'save_24.pth'))
pointnet.load_state_dict(torch.load(savepath +'save_4.pth', map_location=torch.device('cpu')))
pointnet.to(device);

pointnet.eval();

In [ ]:
# check regression
cloud_list = []
label_list = []
output_list = []
predictions_list = []
inputs_list = []
id_list = []
parameter_id = 1
with torch.no_grad():
    tot = 0
    count = 0
    for data in valid_loader:
        inputs, labels, ids = data['pointcloud'].to(device).float(), data['properties'].to(device), data['id'].to(device)
        outputs, __, __ = pointnet(inputs.transpose(1,2))
        print(data['pointcloud'].size(), labels.size(), outputs.size())

        for i in range(outputs.size(0)):
            label_list.append(labels[i][parameter_id].item())
            id_list.append(ids[i].item())
            output_list.append(outputs[i][parameter_id].item())
            predictions_list.append(outputs[i].numpy())
            inputs_list.append(labels[i].numpy())
            cloud_list.append(inputs[i].numpy())
            ratio = ((labels[i][parameter_id]-outputs[i][parameter_id])/labels[i][parameter_id]).item()
            print('r', i+count, ids[i].item(), labels[i][parameter_id].item(), outputs[i][parameter_id].item(), ratio)
            tot += np.absolute(ratio)
            #print('l', labels[i][1].item(), outputs[i][1].item(), ((labels[i][1]-outputs[i][1])/labels[i][1]).item())
        
        count += outputs.size(0)
    print(tot/count)

In [ ]:
label_list, output_list, id_list = np.array(label_list), np.array(output_list), np.array(id_list)

In [ ]:
ratio = np.absolute((label_list - output_list)/label_list)
ratio_ind = ratio.argsort()
id_list = id_list[ratio_ind]

In [ ]:
print(id_list[-10:-1])

In [ ]:
cloud_id = 3
pcd_id = 18718     
pcshow(*cloud_list[cloud_id].T)

In [ ]:
# # save selected cloud
# pcl = o3d.geometry.PointCloud()
# pcl.points = o3d.utility.Vector3dVector(cloud_list[cloud_id])
# o3d.io.write_point_cloud("cloud_"+str(cloud_id)+".pcd", pcl)


In [ ]:
#plot error graph
import matplotlib.pyplot as plt

ratio_neg = (label_list - output_list)/label_list

n, bins, _ = plt.hist(ratio_neg, bins=np.arange(-2,2,0.1))
mid = 0.5*(bins[1:] + bins[:-1])
plt.errorbar(mid, n, yerr=0.01, fmt='none')

In [ ]:
error_threshold = 0.2
correct = ratio[np.where(ratio < error_threshold)]
print(len(ratio), len(correct), len(correct)/len(ratio))


#### Visually analyse predictions

In [ ]:
# visualize predictions side by side with ifc
def visualize_predictions(cloud, element, preds, blueprint):
    ifc = setup_ifc_file(blueprint)
    owner_history = ifc.by_type("IfcOwnerHistory")[0]
    project = ifc.by_type("IfcProject")[0]
    context = ifc.by_type("IfcGeometricRepresentationContext")[0]
    floor = ifc.by_type("IfcBuildingStorey")[0]

    ifc_info = {"owner_history": owner_history,
        "project": project,
       "context": context, 
       "floor": floor}
    
    if element == 'pipe':
        pm = {'r':preds[0], 'l':preds[1], 'd':[preds[2], preds[3], preds[4]] }
        pm['p'] = [-((pm['l']*pm['d'][i])/2) for i in range(3)]
        #print(pm)
        
        create_IfcPipe(pm['r'], pm['l'], pm['d'], pm['p'], ifc, ifc_info)
        
    elif element == 'elbow':
        pm = {'r':preds[0], 'x':preds[1], 'y':preds[2], 'd':[preds[3], preds[4], preds[5]], 'a':preds[6] }
        theta = math.atan(pm['x']/pm['y'])
        pm['axis_dir'] = [math.cos(theta), math.sin(theta)]
        pm['p'] = [0.0, 0.0, 0.0]
        print(pm)
        
        create_IfcElbow(pm['r'], pm['a'], pm['d'], pm['p'], pm['x'],
                        pm['y'], pm['axis_dir'], ifc, ifc_info)
    return vis_ifc_and_cloud(ifc,pcd)

In [ ]:
blueprint = 'data/sample.ifc'
pcd_path = "output/" + cat + "/test/" + str(pcd_id) + ".pcd"
scale_factor = 500
pcd = o3d.io.read_point_cloud(pcd_path)
preds = predictions_list[cloud_id].tolist()
inputs = inputs_list[cloud_id].tolist()
print(preds)
print(inputs)
if cat == 'pipe':
    targets = [0,1]
elif cat == 'elbow':  
    targets = [0,1,2]
    preds[6] *= 200

for i in targets:
    preds[i] = preds[i]*scale_factor


In [ ]:
viewer = visualize_predictions(pcd, cat, preds, blueprint)
viewer
